In [1]:
import pandas as pd


/c/Users/Mason Carnahan/Documents/Bootcamp/Project_1/Project_1


In [2]:
import pandas as pd
import os 
from matplotlib import pyplot as plt
import numpy as np



#Decide what columns we need
A_csv = pd.read_csv('xaa.csv')
B_csv = pd.read_csv('xab.csv', header=None)
C_csv = pd.read_csv('xac.csv', header=None)
D_csv = pd.read_csv('xad.csv', header=None)
#Add correct index to split csv
B_csv.columns = A_csv.columns
C_csv.columns = A_csv.columns
D_csv.columns = A_csv.columns
#Merging
AB_csv = pd.merge(A_csv, B_csv,how='outer')

ABC_csv = pd.merge(AB_csv, C_csv,how='outer')

ABCD_csv = pd.merge(ABC_csv, D_csv,how='outer')

master_df = ABCD_csv

In [3]:
#Function to change GeoLoc value
def geo_switch(geo):
    try:
        [lat, long] = str(geo[0])[1:-1].split(',')
    except TypeError:
        [lat, long] = ['NaN', 'NaN']
    except ValueError:
        [lat, long] = str(geo)[1:-1].split(',')
    
    
    return [lat, long]

In [5]:
#change the master_df using apply
master_df['GeoLocation'] = master_df['GeoLocation'].apply(lambda x: geo_switch(x))

In [91]:
master_df.head()

,Year,StateAbbr,CityName,GeographicLevel,Category,Measure,DataValueTypeID,Data_Value_Type,Data_Value,PopulationCount,GeoLocation,CategoryID,MeasureId
0,2016,US,NaN,US,Prevention,Current lack of health insurance among adults ...,AgeAdjPrv,Age-adjusted prevalence,14.6,308745538,"[NaN, NaN]",PREVENT,ACCESS2
1,2016,US,NaN,US,Prevention,Current lack of health insurance among adults ...,CrdPrv,Crude prevalence,11.6,308745538,"[NaN, NaN]",PREVENT,ACCESS2
2,2016,US,NaN,US,Health Outcomes,Arthritis among adults aged >=18 Years,AgeAdjPrv,Age-adjusted prevalence,23.0,308745538,"[NaN, NaN]",HLTHOUT,ARTHRITIS
3,2016,US,NaN,US,Health Outcomes,Arthritis among adults aged >=18 Years,CrdPrv,Crude prevalence,25.4,308745538,"[NaN, NaN]",HLTHOUT,ARTHRITIS
4,2016,US,NaN,US,Unhealthy Behaviors,Binge drinking among adults aged >=18 Years,AgeAdjPrv,Age-adjusted prevalence,18.0,308745538,"[NaN, NaN]",UNHBEH,BINGE


In [7]:
#unique list of categories and states
cat_id = master_df['Category'].unique()
states = master_df['StateAbbr'].unique()
#dictionary to hold your data
df_dict = {}

#index through to grab dataframe for each category
for index in cat_id:
    category_group = master_df[master_df['Category'] == index]
#get category info for each state        
    for state in states:
        state_group =  category_group[category_group['StateAbbr'] == state]
#create keys for the df_dict and values for each state
        df_dict[index+' '+ state] = state_group

In [8]:
#Prevention dictionary for all states
prevent_dict = {}
for state in states:
#make keys the state abbr
    prevent_dict[state] = (df_dict.get('Prevention'+' '+state))

In [9]:
#Unhealthy dictionary for all states
unhealthy_dict = {}
for state in states: 
#make keys the state abbr
    unhealthy_dict[state] = (df_dict.get('Unhealthy Behaviors'+' '+state))

In [29]:
#Outcomes dictionary for all states
outcomes_dict = {}
for state in states:
#make keys the state abbr
    outcomes_dict[state] = (df_dict.get('Health Outcomes'+' '+state))

In [597]:
outcomes_dict['WY']

,Year,StateAbbr,CityName,GeographicLevel,Category,Measure,DataValueTypeID,Data_Value_Type,Data_Value,PopulationCount,GeoLocation,CategoryID,MeasureId
809617,2016,WY,Cheyenne,City,Health Outcomes,Arthritis among adults aged >=18 Years,AgeAdjPrv,Age-adjusted prevalence,24.6,59466,"[41.1460804265, -104.789064332]",HLTHOUT,ARTHRITIS
809618,2016,WY,Cheyenne,City,Health Outcomes,Arthritis among adults aged >=18 Years,CrdPrv,Crude prevalence,25.9,59466,"[41.1460804265, -104.789064332]",HLTHOUT,ARTHRITIS
809619,2016,WY,Cheyenne,Census Tract,Health Outcomes,Arthritis among adults aged >=18 Years,CrdPrv,Crude prevalence,24.5,4580,"[41.1243595766, -104.813946446]",HLTHOUT,ARTHRITIS
809620,2016,WY,Cheyenne,Census Tract,Health Outcomes,Arthritis among adults aged >=18 Years,CrdPrv,Crude prevalence,23.9,3639,"[41.1100123323, -104.823728426]",HLTHOUT,ARTHRITIS
809621,2016,WY,Cheyenne,Census Tract,Health Outcomes,Arthritis among adults aged >=18 Years,CrdPrv,Crude prevalence,23.3,415,"[41.1209013739, -104.773243747]",HLTHOUT,ARTHRITIS
809622,2016,WY,Cheyenne,Census Tract,Health Outcomes,Arthritis among adults aged >=18 Years,CrdPrv,Crude prevalence,22.9,8090,"[41.1360981311, -104.75799285]",HLTHOUT,ARTHRITIS
809623,2016,WY,Cheyenne,Census Tract,Health Outcomes,Arthritis among adults aged >=18 Years,CrdPrv,Crude prevalence,26.3,5968,"[41.1398184857, -104.783033115]",HLTHOUT,ARTHRITIS
809624,2016,WY,Cheyenne,Census Tract,Health Outcomes,Arthritis among adults aged >=18 Years,CrdPrv,Crude prevalence,25.0,4331,"[41.1347650805, -104.819326156]",HLTHOUT,ARTHRITIS
809625,2016,WY,Cheyenne,Census Tract,Health Outcomes,Arthritis among adults aged >=18 Years,CrdPrv,Crude prevalence,29.0,1835,"[41.1493692774, -104.798429086]",HLTHOUT,ARTHRITIS
809626,2016,WY,Cheyenne,Census Tract,Health Outcomes,Arthritis among adults aged >=18 Years,CrdPrv,Crude prevalence,27.7,2766,"[41.1565244226, -104.833475008]",HLTHOUT,ARTHRITIS


In [125]:
def delete_crude(dataframe):
    new_df = {}
    for state in states:
        fixed_dataframe = dataframe[state][dataframe[state]['Data_Value_Type'] == 'Age-adjusted prevalence']
        new_df[state] = fixed_dataframe
    return new_df

In [667]:
def rank_data(cat_dict):
    rank_dict = {}
    cat_dict = delete_crude(cat_dict)
    
   
    state_dict = {}
    for state in states:
        state_pop = 0
        state_adj_pop = 0
        state_df = cat_dict[state]
        state_population = int(state_df.groupby(['StateAbbr']).sum()['PopulationCount'])
        #state dataframe
        city_dict = {}
        cities_list = state_df['CityName'].unique()
    
        for city in cities_list:
            
            #city dataframe
            city_df = state_df[state_df['CityName'] == city]
            city_population = city_df['PopulationCount'].values[0:1]
            
            #city population (will multiply with DataVal to get measure_count)
            #city_population = city_df['PopulationCount'].drop_duplicates().reset_index(drop=True)
            state_pop = state_pop + city_population
            #list of all measure_id's for city
            measure_list = city_df['MeasureId'].unique()
            
            measure_dict = {}
            for measure in measure_list:
                measure_df = city_df[city_df['MeasureId'] == measure]
                measure_data_val = measure_df['Data_Value'].values[0:1]
                measure_count = city_population*measure_data_val
                measure_dict[measure] = {'Data_Value' : float(measure_data_val),
                                         'Count': float(measure_count)}
            measure_dict['City Pop'] = city_population
            city_dict[city] = measure_dict
        state_dict[state+' Population'] = state_pop
        state_dict[state+' Population- ALT'] = state_population
        state_dict[state] = city_dict
        
        
#        for cities:
#            state_pop += pop
#            state_adj_pop += pop * DataValue
        
            
    
    return state_dict

In [668]:
rank_data(outcomes_dict)

{'US Population': array([], dtype=int64),
 'US Population- ALT': 4013691994,
 'US': {nan: {}},
 'AL Population': array([965304], dtype=int64),
 'AL Population- ALT': 12548952,
 'AL': {'Birmingham': {'ARTHRITIS': {'Data_Value': 29.3,
    'Count': 6218544.100000001},
   'BPHIGH': {'Data_Value': 44.1, 'Count': 9359651.700000001},
   'CANCER': {'Data_Value': 5.7, 'Count': 1209750.9000000001},
   'CASTHMA': {'Data_Value': 10.5, 'Count': 2228488.5},
   'CHD': {'Data_Value': 7.5, 'Count': 1591777.5},
   'COPD': {'Data_Value': 8.7, 'Count': 1846461.9},
   'DIABETES': {'Data_Value': 16.7, 'Count': 3544357.9},
   'HIGHCHOL': {'Data_Value': 35.3, 'Count': 7491966.1},
   'KIDNEY': {'Data_Value': 3.9, 'Count': 827724.2999999999},
   'MHLTH': {'Data_Value': 15.1, 'Count': 3204778.6999999997},
   'STROKE': {'Data_Value': 5.0, 'Count': 1061185.0},
   'PHLTH': {'Data_Value': 16.3, 'Count': 3459463.1},
   'TEETHLOST': {'Data_Value': 22.9, 'Count': 4860227.3}},
  'Hoover': {'ARTHRITIS': {'Data_Value': 23

In [619]:
b.values[0]

59466

US         AL        AZ       AK        AR    CA      CO        CT          DE        DC     FL      GA        NaN
Avondale   Chandler  Gilbert  Glendale  Mesa  Peoria  Phoenix   Scottsdale  Surprise  Tempe  Tucson  Yuma      NaN
ARTHRITIS  BPHIGH    CANCER   CASTHMA   CHD   COPD    DIABETES  HIGHCHOL    KIDNEY    MHLTH  PHLTH   STROKE    NaN
dtype: object

In [210]:
cities = ['Avondale', 'Chandler', 'Gilbert', 'Glendale', 'Mesa', 'Peoria',
       'Phoenix', 'Scottsdale', 'Surprise', 'Tempe', 'Tucson', 'Yuma']
meas = ['ARTHRITIS', 'BPHIGH', 'CANCER', 'CASTHMA', 'CHD', 'COPD',
       'DIABETES', 'HIGHCHOL', 'KIDNEY', 'MHLTH', 'PHLTH', 'STROKE',
       'TEETHLOST']